In [1]:
from pathlib import Path
import pydicom
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [7]:
train_data_dir = Path("/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images")
csv_file = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
labels = pd.read_csv(csv_file)
labels.head(6)

In [5]:
labels = labels.drop_duplicates("patientId")

# duplicates patient id's deleted

In [8]:
save_path = Path("processed")

In [13]:
fig,axis = plt.subplots(3,3, figsize =(9,9))
c = 0
for i in range(3):
    for j in range(3):
        patient_id = labels.patientId.iloc[c]
        dcm_path = train_data_dir/patient_id
        dcm_path = dcm_path.with_suffix(".dcm")
        dcm = pydicom.read_file(dcm_path).pixel_array
        
        label = labels["Target"].iloc[c]
        
        axis[i][j].imshow(dcm,cmap ="bone")
        axis[i][j].set_title(label)
        c +=1

In [15]:
sums, sums_squared = 0,0

for c, patient_id in enumerate(tqdm(labels.patientId)):
        patient_id = labels.patientId.iloc[c]
        dcm_path = train_data_dir/patient_id
        dcm_path = dcm_path.with_suffix(".dcm")
        dcm = pydicom.read_file(dcm_path).pixel_array/255
        
        dcm_array = cv2.resize(dcm, (224,224)).astype(np.float16)
        
        label = labels.Target.iloc[c]
        
        train_or_val = "train" if c < 24000 else "val"
        
        current_save_path = save_path/train_or_val/str(label)
        current_save_path.mkdir(parents = True, exist_ok = True)
        np.save(current_save_path/patient_id, dcm_array)
        
        normalizer = 224*224
        
        if train_or_val == "train":
            sums += np.sum(dcm_array) / normalizer
            sums_squared += (dcm_array ** 2).sum() / normalizer

In [17]:
mean = sums/24000
std = np.sqrt((sums_squared / 24000) - mean **2)
mean, std